<a href="https://colab.research.google.com/github/NeginMirian/Rag-implementation/blob/main/Rag_imp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# !pip install chromadb
# !pip install transformers
# !pip install huggingface-hub
# !pip install sentence_transformers
# !pip install ipywidgets
import pandas as pd
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer
from huggingface_hub import InferenceClient
import numpy as np
import os

In [6]:
def text_embedding(text) -> None:
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    return model.encode(text)

In [7]:
def generate_context(query):
    vector=text_embedding(query).tolist()

    results=collection.query(
        query_embeddings=vector,
        n_results=15,
        include=["documents"]
    )

    res = "\n".join(str(item) for item in results['documents'][0])
    return res

In [8]:
def chat_completion(system_prompt, user_prompt,length=1000):
    final_prompt=f"""<s>[INST]<<SYS>>
    {system_prompt}
    <</SYS>>

    {user_prompt} [/INST]"""
    return client.text_generation(prompt=final_prompt,max_new_tokens = length).strip()

In [15]:
#data
df=pd.read_csv('/the_grammy_awards.csv')
df=df.loc[df['year'] == 2019]
df=df.dropna(subset=['nominee'])
df.loc[:, 'category'] = df['category'].str.lower()
df.loc[:, 'text'] = df['artist'] + ' got nominated under the category, ' + df['category'] + ', for the track ' + df['nominee'] + ' to win the award'
df.loc[df['winner'] == False, 'text'] = df['artist'] + ' got nominated under the category, ' + df['category'] + ', for the track ' + df['nominee'] + ' but did not win'


In [24]:
df.head()

,year,title,published_at,updated_at,category,nominee,artist,workers,img,winner,text
0,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,record of the year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Fi...",https://www.grammy.com/sites/com/files/styles/...,True,Billie Eilish got nominated under the category...
1,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,record of the year,"Hey, Ma",Bon Iver,"BJ Burton, Brad Cook, Chris Messina & Justin V...",https://www.grammy.com/sites/com/files/styles/...,True,"Bon Iver got nominated under the category, rec..."
2,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,record of the year,7 rings,Ariana Grande,"Charles Anderson, Tommy Brown, Michael Foster ...",https://www.grammy.com/sites/com/files/styles/...,True,Ariana Grande got nominated under the category...
3,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,record of the year,Hard Place,H.E.R.,"Rodney “Darkchild” Jerkins, producer; Joseph H...",https://www.grammy.com/sites/com/files/styles/...,True,"H.E.R. got nominated under the category, recor..."
4,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,record of the year,Talk,Khalid,"Disclosure & Denis Kosiak, producers; Ingmar C...",https://www.grammy.com/sites/com/files/styles/...,True,"Khalid got nominated under the category, recor..."


In [27]:
df['text'] = df['text'].astype(str)

# Now create the 'docs' and 'ids' lists
docs = df["text"].tolist()
ids = [str(x) for x in df.index.tolist()]


In [28]:
client = chromadb.Client()
collection = client.get_or_create_collection("Grammy-2019")

In [26]:
print(collection)


name='Grammy-2019' id=UUID('dfc786cb-3fac-43e1-86ee-d89403bfc30f') metadata=None tenant='default_tenant' database='default_database'


In [29]:
docs=df["text"].tolist()
ids= [str(x) for x in df.index.tolist()]
collection.add(
    documents=docs,
    ids=ids
)

In [30]:
URI='http://139.84.142.100:8080'
client = InferenceClient(model=URI)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Neuer Abschnitt

In [31]:
#query="What did Ke Huy Quan work on?"
#query="Which movie won the best music award?"
query="Did Billie Eilish won a reward for grammy 2019?"
#query="Who is the music director of RRR?"
context=generate_context(query)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [32]:
system_prompt="""\
You are a helpful AI assistant that can answer questions on grammy 2019 awards. Answer based on the context provided. If you cannot find the correct answerm, say I don't know. Be concise and just include the response.
"""

In [33]:
user_prompt=f"""
Based on the context:
{context}
Answer the below query:
{query}
"""

In [36]:
user_prompt

'\nBased on the context:\nBillie Eilish got nominated under the category, album of the year, for the track When We All Fall Asleep, Where Do We Go? to win the award\nBillie Eilish got nominated under the category, best pop vocal album, for the track When We All Fall Asleep, Where Do We Go? to win the award\nBillie Eilish got nominated under the category, record of the year, for the track Bad Guy to win the award\nBillie Eilish got nominated under the category, best pop solo performance, for the track Bad Guy to win the award\nH.E.R. got nominated under the category, album of the year, for the track I Used To Know Her to win the award\nH.E.R. got nominated under the category, record of the year, for the track Hard Place to win the award\nLizzo got nominated under the category, album of the year, for the track Cuz I Love You (Deluxe) to win the award\nLana Del Rey got nominated under the category, album of the year, for the track Norman F***ing Rockwell! to win the award\nEd Sheeran got 

In [37]:
chat_completion(system_prompt,user_prompt)

ConnectTimeout: (MaxRetryError("HTTPConnectionPool(host='139.84.142.100', port=8080): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7ab2c16e1990>, 'Connection to 139.84.142.100 timed out. (connect timeout=None)'))"), '(Request ID: 789394a9-48fc-4d4e-be15-3a357134dec1)')

In [38]:
!git config --global user.name "NeginMirian"
!git config --global user.email "negin.mirian@gmail.com"

In [39]:
!git clone https://github.com/NeginMirian/Rag-implementation.git

Cloning into 'Rag-implementation'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), done.


In [43]:
%cd Rag-implementation
!git add .
!git commit -m "Add  Rag script from Colab"

!git push origin main

/content/Rag-implementation
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
fatal: could not read Username for 'https://github.com': No such device or address
